In [1]:
import pandas as pd
import requests
import datetime
from geopy.geocoders import OpenMapQuest

In [3]:
sites_location=pd.read_csv('../data/sites_location.csv', delimiter =';')

In [54]:
sites_location.head()

,site_id,City,UTC_offset,lat_lon
0,0,Jacksonville,-4,"(30.3321838, -81.655651)"
1,1,London,0,"(51.5073219, -0.1276474)"
2,2,Phoenix,-7,"(33.4485866, -112.0773456)"
3,3,Philadelphia,-4,"(39.9523993, -75.1635899)"
4,4,San Francisco,-7,"(37.7647993, -122.4629897)"


In [52]:
from geopy.geocoders import OpenMapQuest
def location(my_address):
    geolocator = OpenMapQuest(api_key='LfGsouoBmQGZZAGpkFZzXGyzo27zOFK2')
    location = geolocator.geocode(my_address, timeout=10)
    return (location.latitude, location.longitude)
sites_location['lat_lon']=sites_location['City'].apply(location)

In [62]:
from timezonefinder import TimezoneFinder
def get_timezone(addr):
    tf = TimezoneFinder()
    return tf.timezone_at(lng=addr[1], lat=addr[0])
sites_location['timezone']=sites_location['lat_lon'].apply(get_timezone)
sites_location.head()

,site_id,City,UTC_offset,lat_lon,timezone
0,0,Jacksonville,-4,"(30.3321838, -81.655651)",America/New_York
1,1,London,0,"(51.5073219, -0.1276474)",Europe/London
2,2,Phoenix,-7,"(33.4485866, -112.0773456)",America/Phoenix
3,3,Philadelphia,-4,"(39.9523993, -75.1635899)",America/New_York
4,4,San Francisco,-7,"(37.7647993, -122.4629897)",America/Los_Angeles


In [9]:
weather = pd.read_csv('../data/weather_train.csv')
weather.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [63]:
df = pd.merge(weather, sites_location[['site_id', 'UTC_offset', 'timezone']], on = 'site_id', how = 'left')
df.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,UTC_offset,timezone
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,-4,America/New_York
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5,-4,America/New_York
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0,-4,America/New_York
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0,-4,America/New_York
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6,-4,America/New_York


In [68]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [85]:
df['timestamp_adjusted'] = df.groupby('timezone')['timestamp'].transform(lambda x: x.dt.tz_localize('UTC').dt.tz_convert(x.name).dt.tz_localize(None))
df.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,UTC_offset,timezone,timestamp_adjusted
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,-4,America/New_York,2015-12-31 19:00:00
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5,-4,America/New_York,2015-12-31 20:00:00
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0,-4,America/New_York,2015-12-31 21:00:00
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0,-4,America/New_York,2015-12-31 22:00:00
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6,-4,America/New_York,2015-12-31 23:00:00


In [86]:
weather['timestamp'] = df['timestamp_adjusted']
weather.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2015-12-31 19:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2015-12-31 20:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2015-12-31 21:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2015-12-31 22:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2015-12-31 23:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [87]:
weather.to_csv('../data/weather_train_tz_fix.csv', index=False)